{/* cspell:ignore webkitallowfullscreen allowfullscreen frameborder referrerpolicy Lipinska Pellow Jarman Tilly Ardle Expressibility Arrasmith */}

# Classical Optimizers

## What is an optimizer?

Victoria Lipinska tells us about classical optimizers, and how they function as part of VQE.

You will hear about a few example optimizers and how they perform in the presence and absence of noise.

<iframe width="690" height="390" src="https://video.ibm.com/embed/recorded/132414916"></iframe>

**References**

- [A Comparison of Various Classical Optimizers for a Variational Quantum Linear Solver, Pellow-Jarman, et al.](https://arxiv.org/pdf/2106.08682.pdf)
- [The Variational Quantum Eigensolver: A review of methods and best practices, Tilly, et al.](https://arxiv.org/abs/2111.05176)
- [Quantum computational chemistry, McArdle, et al.](https://arxiv.org/pdf/1808.10402.pdf)
- [Barren plateaus in quantum neural network training landscapes, McClean, et al.](https://arxiv.org/pdf/1803.11173.pdf)
- [Connecting Ansatz Expressibility to Gradient Magnitudes and Barren Plateaus, Holmes, et al.](https://journals.aps.org/prxquantum/pdf/10.1103/PRXQuantum.3.010313)
- [Effect of barren plateaus on gradient-free optimization, Arrasmith, et al.](https://arxiv.org/pdf/2011.12245.pdf)